## Search API
This notebook explains how to use the API to search the database using either search terms or a set of bibcodes. Lists of query parameters and search fields are also given.

The base URL (denoted as {base_url} in the examples below) for queries is `https://api.adsabs.harvard.edu/v1/search`. 

All queries to the API must include an HTTP header that specifies your API access token, e.g.

`curl -H 'Authorization: Bearer <token>' 'https://api.adsabs.harvard.edu/v1/search/query?q=star'`

In the examples below, {token_header} is used to denote `-H 'Authorization: Bearer <token>'`, where `<token>` is your personal API access token as described in the top-level [README](https://github.com/adsabs/adsabs-dev-api/blob/master/README.md).

Notes:
* All examples use the default HTTP method GET, unless otherwise specified.
* the API examples in this notebook generally assume you're using `curl` statements from the shell command line--you'll see `%%bash` at the top of a cell if its contents should be run in the shell instead of in Python. Guidelines for translating bash `curl` statements into Python can be found in the [converting curl to python notebook](Converting_curl_to_python.ipynb). If you prefer, you can use the unofficial Python package instead; see the top-level [README](https://github.com/adsabs/adsabs-dev-api/blob/master/README.md) for more information.

### Contents
- [Search API](#Search-API)
- [Query parameters](#Query-Parameters)
- [Fields](#Fields)
- [Examples](#Examples)
- [Advanced search syntax](#Advanced-Search-Syntax)

### Get search results
`curl {token_header} {base_url}/query?q=search+term&fl=field1,field2`

To search, construct a URL using the base URL given [above](#search-api), along with your search parameters as detailed in the [Query Parameters](#query-parameters) section below. All text must be UTF-8 and URL encoded. For example:

In [1]:
import sys

v = sys.version

# accented letters, special characters, and spaces need to be encoded
# Python has a package for this, though the functionality is slightly different between python 2 and python 3
if int(v[0]) == 2:
    import urllib
    query = "author:martínez neutron star"
    encoded_query = urllib.quote_plus(query)
else:
    from urllib.parse import urlencode, quote_plus
    query = {"author":"martínez neutron star"}
    encoded_query = urlencode(query,quote_via=quote_plus)
print(encoded_query)

# note that the colon (:) may be encoded, depending on the algorithm you use. Your curl request 
# should accept either the unencoded colon (:) or the encoded version (%3A)

author%3Amart%C3%ADnez+neutron+star


The response from your `curl` statement will be JSON encoded. 

Also, note that the search API uses the same syntax as [Apache Solr](http://lucene.apache.org/solr/). For a full reference of query possibilities, please refer to the Solr documentation and [ADS Search Help](http://adsabs.github.io/help/search/). Later sections in this notebook present useful parameters and patterns for the vast majority of use cases, but are not meant to be exhaustive.

Note: to print the JSON response in a more readable manner, add `| python -m json.tool` to the end of your `curl` statement.

In [2]:
%%bash
token="your-token-here"
curl -H "Authorization: Bearer $token" \
    "https://api.adsabs.harvard.edu/v1/search/query?q=author%3Amart%C3%ADnez+neutron+star&fl=author&rows=1" \
    | python -m json.tool

{
    "response": {
        "docs": [
            {
                "author": [
                    "P\u00e9rez Rojas, Hugo",
                    "Mart\u00ednez, Aurora P\u00e9rez",
                    "Mesquita, Alexandre",
                    "Razeira, Mois\u00e9s",
                    "Gomes, Rosana O.",
                    "Vasconcellos, C\u00e9sar A. Z."
                ]
            }
        ],
        "numFound": 168,
        "start": 0
    },
    "responseHeader": {
        "QTime": 743,
        "params": {
            "fl": "author",
            "q": "author:mart\u00ednez neutron star",
            "rows": "1",
            "start": "0",
            "wt": "json",
            "x-amzn-trace-id": "Root=1-5b354e2a-a5b6ddca2051f3311a0beb8f"
        },
        "status": 0
    }
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   398  100   398    0     0    398      0  0:00:01  0:00:01 --:--:--   366


### Parse a query
`curl {token_header} {base_url}/qtree?q=termORother+term` 

Use the syntax above to return a query tree, also known as an Abstract Syntax Tree (AST), as understood by our query parser. This can be useful if you want to modify and/or enhance queries.

In [3]:
%%bash
token="your-token-here"
curl -H "Authorization: Bearer $token" \
    "https://api.adsabs.harvard.edu/v1/search/qtree?q=author%3Amart%C3%ADnez+neutron+star"

{"responseHeader":{"status":0,"QTime":3,"params":{"q":"author:martínez neutron star","x-amzn-trace-id":"Root=1-5b2d4f71-c818c6a8dfcab0d0fc622eba","fl":"id","start":"0","rows":"10","wt":"json"}},"qtree":"\n{\"name\":\"OPERATOR\", \"label\":\"DEFOP\", \"children\": [\n    {\"name\":\"MODIFIER\", \"label\":\"MODIFIER\", \"children\": [\n        {\"name\":\"TMODIFIER\", \"label\":\"TMODIFIER\", \"children\": [\n            {\"name\":\"FIELD\", \"label\":\"FIELD\", \"children\": [\n                {\"name\":\"TERM_NORMAL\", \"input\":\"author\", \"start\":0, \"end\":5},\n                {\"name\":\"QNORMAL\", \"label\":\"QNORMAL\", \"children\": [\n                    {\"name\":\"TERM_NORMAL\", \"input\":\"martínez\", \"start\":7, \"end\":14}]\n                }]\n            }]\n        }]\n    },\n    {\"name\":\"MODIFIER\", \"label\":\"MODIFIER\", \"children\": [\n        {\"name\":\"TMODIFIER\", \"label\":\"TMODIFIER\", \"children\": [\n            {\"name\":\"FIELD\", \"label\":\"FIELD

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1705  100  1705    0     0   1705      0  0:00:01 --:--:--  0:00:01  7478


### Post a large identifier query
`method = POST`

`curl -H "Content-Type: big-query/csv" {token_header} {base_url}/bigquery?{urlparams} -X POST -d {payload}`

This type of query, typically called a "big query" for short, returns [standard search results](#get-search-results), but instead of the typical search terms, accepts as its input a list of bibcodes. There is a maximum of 2000 bibcodes allowed for each big query request. There are also limits on how often you can make a big query request; typically, only 100 requests per day per user are allowed.

Name          | Type   | Description 
:-------------|:------:|:--------------
`urlparams`   |`string`| **Required**. Query parameters must be encoded as in a [standard query](#get-search-results). The query parameter `q=` must be included even if no additional searching is desired; to prevent it from restricting the search results, enter `q=*:*`. It's also recommended to use the `fl=` parameter to specify which fields to return.
`payload`     |`string`| **Required**. Newline (`\n`) separated list of bibcodes to return. The first line specifies the field that will be used for searching, i.e. `bibcode` (for example, `bibcode\n1907AN....174...59.\n1908PA.....16..445.\n1989LNP...334..242S`)

Notes:
- Currently, we only allow searching in the `bibcode` field. You can submit both canonical bibcodes (the ones that appear on a paper's abstract page, or in the URL pointing to the abstract page) as well as alternate bibcodes (e.g. bibcodes that pointed to a previous version of a paper, such as the arXiv version). The search automatically recognizes both types.
- The list of bibcodes supplied in the `payload` is *only applied after the main search*, specified with the `q=` parameter. To return the entire list of supplied bibcodes, use `q=*:*`.
- Currently, the default number of results returned by the big query is not controlled by the number of bibcodes requested, but rather is set to the default value of 10. To adjust this, set `&rows=` either to the number of bibcodes requested or to the maximum value of 2000.


In [4]:
%%bash
token="your-token-here"
curl -H "Content-Type: big-query/csv" -H "Authorization: Bearer $token" \
    "https://api.adsabs.harvard.edu/v1/search/bigquery?q=*:*&fl=bibcode,title&rows=2000" \
    -X POST -d $'bibcode\n1907AN....174...59.\n1908PA.....16..445.\n1989LNP...334..242S'


{"responseHeader":{"status":0,"QTime":22,"params":{"q":"*:*","x-amzn-trace-id":"Root=1-5b2d4f73-0ac58ac0850c2bcc3104f85e","fl":"bibcode,title","start":"0","fq":"{!bitset}","rows":"2000","wt":"json"}},"response":{"numFound":3,"start":0,"docs":[{"bibcode":"1908PA.....16..445.","title":["Variable Stars"]},{"bibcode":"1907AN....174...59.","title":["Kleine Mitteilungen"]},{"bibcode":"1989LNP...334..242S","title":["The Optical and Radio Properties of X-Ray Selected Bl-Lacertae Objects"]}]}}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   557  100   490  100    67    490     67  0:00:01 --:--:--  0:00:01  3396


## Query Parameters
All query parameters passed in the search URL must be UTF-8, URL-encoded strings. Due to the requirements of the authentication library used for validating requests, most non-ASCII characters that appear in the URL need to be encoded; for example, the double quote character (") should be encoded as %22. In most programming languages, such as Python, the libraries used to retrieve content from web services will do this encoding for you; however, if you're using `curl` requests in the shell, such as shown in the examples in this notebook, you'll need to perform this encoding yourself.

#### q
**Required** The search query. This should be a UTF-8, URL-encoded string of <=1000 characters. `q` accepts both fields (`title:exoplanets`) and unfielded (`exoplanets`) searches.

#### rows 
The number of results to return. The default is 10 and the maximum is 2000.

#### start
The starting point for returned results, used for pagination. The default is 0. To return the next page of results, set `start` equal to the value of `start` from the previous request, plus the number of results returned in the previous request. For the default values, set `start=10` to return the second page of results.

#### fl
The list of fields to return. The value should be a comma separated list of field names, e.g. `fl=bibcode,author,title`. The default is the document id (`fl=id`). A non-exhaustive list of available fields [is shown below](#fields).

#### fq
Filters the list of search results. The syntax is the same as that for the `q` parameter. Adding search parameters via the `fq` parameter can speed up search results, as it searches only the results returned by the search entered via the `q` parameter, not the entire index. This parameter may be used more than once in a single search URL.

#### sort
The sorting field and direction to be used when returning results. The format requires both the field to sort on (see [the list below](#fields) and the direction, either `asc` or `desc` (ascending or descending). For example, an appropriately formatted sort parameter is `sort=citation_count+desc`. The default sort method is the relevancy score as calculated by the search engine. Other useful fields to sort on may be `date`, `read_count`, `first_author`, or `bibcode`.

## Fields
This is a non-exhaustive list of fields available for searching via the API. A more comprehensive list is available in our [help pages](https://adsabs.github.io/help/search/comprehensive-solr-term-list). These fields can be used with the `fl` and `sort` parameters. 

Note: some fields, such as body, can be searched but not returned via `fl` or sorted on. Also, multivalued fields, such as author, cannot be used for sorting.

* `abstract` - the abstract of the record
* `ack` - the acknowledgements section of an article
* `aff` - an array of the authors' affiliations
* `alternate_bibcode` - list of alternate bibcodes for a single record; if a published record also has an arXiv version, the bibcode associated with the arXiv version is here
* `alternate_title` - list of alternate titles for a single record (usually if they are in multiple languages)
* `arxiv_class` - the arXiv class the pre-print was submitted to
* `author` - an array of the author names associated with the record
* `bibcode` - the canonical ADS bibcode identifier for this record
* `bibgroup` - the bibliographic groups that the bibcode has been associated with
* `bibstem` - the abbreviated name of the journal or publication, e.g., *ApJ*.
* `body`\* - the full text content of the article
* `citation_count` - number of citations the item has received
* `copyright` - the copyright applied to the article
* `data` - the list of sources that have data related to this bibcode
* `date` - the machine-readable version of `pubdate`, useful for sorting
* `database` - database the record is associated with (astronomy, physics, or general). By default, all three databases are searched; to limit to astronomy articles, add `fq=database:astronomy` to the URL
* `doi`-  the digital object identifier of the article
* `doctype` - the type of document it is (see [here](https://adsabs.github.io/help/search/search-syntax) for a list of doctypes)
* `first_author` - the first author of the article
* `grant` - the list of grant IDs and agencies noted within an article
* `id` - a **non-persistent** unique integer for this record, used for fast look-up of a document
* `identifier` - an array of alternative identifiers for the record. May contain alternate bibcodes, DOIs and/or arxiv ids.
* `indexstamp` - time at which the document was (last) indexed
* `issue` - issue the record appeared in
* `keyword` - an array of normalized and un-normalized keyword values associated with the record
* `lang`\* - the language of the article's title
* `orcid_pub` - ORCiD iDs supplied by publishers
* `orcid_user` - ORCiD iDs supplied by knonwn users in the ADS
* `orcid_other` - ORCiD iDs supplied by anonymous users in the ADS
* `page` - starting page
* `property` - an array of miscellaneous flags associated with the record (see [here](https://adsabs.github.io/help/search/search-syntax) for a list of properties
* `pub` - the canonical name of the publication the record appeared in
* `pubdate` - publication date in the form YYYY-MM-DD (DD value will always be "00"). This field is stored as a human-readable string, so is useful for being returned via `fl`, but not for sorting (see `date`)
* `read_count` - number of times the record has been viewed within in a 90-day windows (ads and arxiv)
* `title` - the title of the record
* `vizier` - the subject tags given to the article by VizieR
* `volume` - volume the record appeared in
* `year` - the year the article was published

\* *These fields are only indexed and so are only searchable - they are not stored or returned by the search end point, if requested*

## Examples
Example `curl` requests.

In [5]:
%%bash
token="your-token-here"
# search by bibcode, return the title
curl -H "Authorization: Bearer $token" \
    "https://api.adsabs.harvard.edu/v1/search/query?q=bibcode:2011ApJ...737..103S&fl=title"

{"responseHeader":{"status":0,"QTime":6,"params":{"q":"bibcode:2011ApJ...737..103S","x-amzn-trace-id":"Root=1-5b2d4f77-eb5609cc57124c307500906c","fl":"title","start":"0","rows":"10","wt":"json"}},"response":{"numFound":1,"start":0,"docs":[{"title":["Measuring Reddening with Sloan Digital Sky Survey Stellar Spectra and Recalibrating SFD"]}]}}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   344  100   344    0     0    344      0  0:00:01 --:--:--  0:00:01  1702


In [6]:
%%bash
token="your-token-here"
# search in all metadata fields for "black holes", restricted to the astronomy database; return a list of bibcodes
curl -H "Authorization: Bearer $token" \
    "https://api.adsabs.harvard.edu/v1/search/query?q=black+holes&fq=database:astronomy&fl=bibcode"

{"responseHeader":{"status":0,"QTime":977,"params":{"q":"black holes","x-amzn-trace-id":"Root=1-5b2d4f78-87a695a096d4ece4e36e76c6","fl":"bibcode","start":"0","fq":"database:astronomy","rows":"10","wt":"json"}},"response":{"numFound":60233,"start":0,"docs":[{"bibcode":"2004IAUS..222..115T"},{"bibcode":"2006kbls.confE..15H"},{"bibcode":"1997PKAS...12..145P"},{"bibcode":"1992ianp.symp..225S"},{"bibcode":"2017APS..APR.E5009F"},{"bibcode":"2017nova.pres.2298K"},{"bibcode":"1986Sci...234..882T"},{"bibcode":"2002PhRvD..66f3505B"},{"bibcode":"1996PhRvL..77.5164K"},{"bibcode":"2017ApJ...840L..24F"}]}}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   600  100   600    0     0    600      0  0:00:01  0:00:01 --:--:--   528


In [7]:
%%bash
token="your-token-here"
# search in all metadata fields for "dark energy", filter by author, return a list of bibcodes and citation counts, 
# sorted by citation count (the double quotes are encoded as %22)
curl -H "Authorization: Bearer $token" \
    "https://api.adsabs.harvard.edu/v1/search/query?q=%22dark+energy%22&fq=author:%22Spergel,+D%22&sort=citation_count+desc&fl=bibcode,citation_count"
    

{"responseHeader":{"status":0,"QTime":223,"params":{"q":"\"dark energy\"","x-amzn-trace-id":"Root=1-5b2d4f7a-026b88865a0439add4cb3005","fl":"bibcode,citation_count","start":"0","sort":"citation_count desc","fq":"author:\"Spergel, D\"","rows":"10","wt":"json"}},"response":{"numFound":62,"start":0,"docs":[{"citation_count":8728,"bibcode":"2003ApJS..148..175S"},{"citation_count":6436,"bibcode":"2007ApJS..170..377S"},{"citation_count":6397,"bibcode":"2011ApJS..192...18K"},{"citation_count":4484,"bibcode":"2009ApJS..180..330K"},{"citation_count":3976,"bibcode":"2003ApJS..148....1B"},{"citation_count":1581,"bibcode":"2009ApJS..180..306D"},{"citation_count":1383,"bibcode":"2009ApJS..180..225H"},{"citation_count":1209,"bibcode":"2010MNRAS.401.2148P"},{"citation_count":478,"bibcode":"2010MNRAS.404...60R"},{"citation_count":437,"bibcode":"2011ApJS..192...17B"}]}}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   866  100   866    0     0    866      0  0:00:01 --:--:--  0:00:01  2086


In [8]:
%%bash
token="your-token-here"
# limit a search to only refereed articles
curl -H "Authorization: Bearer $token" \
    "https://api.adsabs.harvard.edu/v1/search/query?q=author:%22Kurtz,+M%22&fq=property:refereed&fl=title"

{"responseHeader":{"status":0,"QTime":68,"params":{"q":"author:\"Kurtz, M\"","x-amzn-trace-id":"Root=1-5b2d4f7b-beaf5e3c449be466ef2fb630","fl":"title","start":"0","fq":"property:refereed","rows":"10","wt":"json"}},"response":{"numFound":116,"start":0,"docs":[{"title":["Correlated radon and CO<SUB>2</SUB> variations near the San Andreas Fault"]},{"title":["Methanol synthesis over ZnO: A structure-sensitive reaction?"]},{"title":["Effects of Hyperbaric Hypocapnic Ventilation with One Hundred Per Cent Oxygen and Tham on Anoxia by Tracheal Occlusion *"]},{"title":["Multiple pass laser interferometer"]},{"title":["Electron impact ionization in the vicinity of comets"]},{"title":["Measuring the Dark Matter Scale of Local Group Dwarf Spheroidals"]},{"title":["Zaragozic acids: a family of fungal metabolites that are picomolar competitive inhibitors of squalene synthase."]},{"title":["A cognitive system for astronomical image interpretation"]},{"title":["Sphingoid Base 1-phosphate Phosphatase: 

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1189  100  1189    0     0   1189      0  0:00:01 --:--:--  0:00:01  3898


In [9]:
%%bash
token="your-token-here"
# search for the phrase "transiting exoplanets", fetch 5 rows
curl -H "Authorization: Bearer $token" \
    "https://api.adsabs.harvard.edu/v1/search/query?q=%22transiting+exoplanets%22+&rows=5&fl=bibcode&sort=bibcode+desc"

{"responseHeader":{"status":0,"QTime":137,"params":{"q":"\"transiting exoplanets\" ","x-amzn-trace-id":"Root=1-5b2d4f7c-4920a2362a8bb94865f5483c","fl":"bibcode","start":"0","sort":"bibcode desc","rows":"5","wt":"json"}},"response":{"numFound":1631,"start":0,"docs":[{"bibcode":"2018sptz.prop14131M"},{"bibcode":"2018sptz.prop14059B"},{"bibcode":"2018sptz.prop14047D"},{"bibcode":"2018Sci...359.1453C"},{"bibcode":"2018RNAAS...2b..40M"}]}}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   439  100   439    0     0    439      0  0:00:01 --:--:--  0:00:01   993


In [10]:
%%bash
token="your-token-here"
# same, but return the next 5 rows
curl -H "Authorization: Bearer $token" \
    "https://api.adsabs.harvard.edu/v1/search/query?q=%22transiting+exoplanets%22&rows=5&start=5&fl=bibcode&sort=bibcode+desc"

{"responseHeader":{"status":0,"QTime":142,"params":{"q":"\"transiting exoplanets\"","x-amzn-trace-id":"Root=1-5b2d503c-893f6dcad5341f72c5f8fa5a","fl":"bibcode","start":"5","sort":"bibcode desc","rows":"5","wt":"json"}},"response":{"numFound":1631,"start":5,"docs":[{"bibcode":"2018RNAAS...2b..20E"},{"bibcode":"2018RNAAS...2a..47J"},{"bibcode":"2018RNAAS...2a..28L"},{"bibcode":"2018PASP..130g4401B"},{"bibcode":"2018PASP..130f4502T"}]}}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   438  100   438    0     0    438      0  0:00:01 --:--:--  0:00:01  1575


## Advanced Search Syntax
The `q` parameter supports both fielded and unfielded searches:
- `black holes`
- `title:exoplanets`

Use quotation marks to indicate phrase searching (remember to encode quotes and spaces in `curl` statements; double quotes are encoded as %22, while spaces can be encoded as `+`)
- `"black holes"`
- `title:"dark energy"`

Prepend terms with `+` or `-` to indicate inclusion or exclusion (note that after encoding spaces in the search string, the URL may include `++` or `+-`):
- `"transiting exoplanets" -Kepler`
- `"transiting exoplanets" +JWST`

In [11]:
%%bash
token="your-token-here"
curl -H "Authorization: Bearer $token" \
    "https://api.adsabs.harvard.edu/v1/search/query?q=%22transiting+exoplanets%22++JWST&fl=title&rows=3"

{"responseHeader":{"status":0,"QTime":120,"params":{"q":"\"transiting exoplanets\"  JWST","x-amzn-trace-id":"Root=1-5b2d4f81-06c03436981012dfca638861","fl":"title","start":"0","rows":"3","wt":"json"}},"response":{"numFound":150,"start":0,"docs":[{"title":["Transiting Exoplanets with JWST"]},{"title":["Spectroscopy of transiting exoplanets: From Spitzer to JWST"]},{"title":["Characterizing Transiting Exoplanet Atmospheres with JWST"]}]}}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   441  100   441    0     0    441      0  0:00:01 --:--:--  0:00:01  1373


To filter by a publication date range, you can use either the `year` or the `pubdate` fields:
- `pubdate:[2013-07-00 TO *]`
- `pubdate:[2005-01 TO 2007-01]`
- `pubdate:2013-02`
- `year:2013`
- `year:[2012 TO 2013]`

The default search uses a boolean `AND` between terms, but you may use `OR` and `AND` in combination with `()` to create more complex queries.

Prefix queries (wildcards, `*`) are supported for most fields.

## Highlighting search terms
Snippets of text, with search terms highlighted, may be returned by a search query. Limits are in place to prevent scraping of our full-text holdings, as noted below, but more highlights can be returned via these API calls than are displayed in the search results on the website.

The following parameters control the highlights feature:
- `hl=true` - (required) turn on the highlighting feature
- `hl.fl` - (required) a comma-separated list of fields to return highlights for (e.g. title, abstract, or body)
- `hl.snippets` - number of highlighted snippets to return; the maximum is 4
- `hl.fragsize` - length of snippet to return; the maximum length is 100 characters. The snippet positioning is optimized to break on word and sentence boundaries, so the highlighted term will not always be centered in the snippet
- `hl.maxAnalyzedChars` - this parameter is only needed if the highlighted field is very long and the frequency of the search term is low. By default, the first 51,200 characters of a field are used for highlights. If the search is returning fewer snippets than expected, try increasing this value to capture more of the field. However, increasing this value unnecessarily will slow searching

## Downloading PDFs
PDFs may be downloaded using a GET request to the `link_gateway` endpoint. The URL to query consists of the base URL `https://ui.adsabs.harvard.edu/link_gateway/`, the bibcode to retrieve the PDF for, and an endpoint to indicate whether to download the arXiv PDF (`/EPRINT_PDF`) or the publisher PDF (`PUB_PDF`). In addition to the usual header that contains your API token, the following flags will also need to be set:
* `-L` - this flag follows the redirect URL to the arXiv or to the publisher's site
* `-o /full/path/to/output.pdf` - this flag and the required path parameter that follows will download the PDF to the specified location

In [12]:
%%bash
token="your-token-here"
# download the arXiv PDF
curl -H "Authorization: Bearer $token" \
 'https://ui.adsabs.harvard.edu/link_gateway/1999ApJ...517..565P/EPRINT_PDF' \
 -L -o '/full/path/to/output.pdf'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   281  100   281    0     0    281      0  0:00:01  0:00:01 --:--:--   163
100   305  100   305    0     0    305      0  0:00:01  0:00:01 --:--:--   305
100   286  100   286    0     0    143      0  0:00:02  0:00:02 --:--:--   143
100  553k  100  553k    0     0   184k      0  0:00:03  0:00:03 --:--:-- 1641k


In [13]:
%%bash
token="your-token-here"
# download the publisher PDF
curl -H "Authorization: Bearer $token" \
 'https://ui.adsabs.harvard.edu/link_gateway/1998AJ....116.1009R/PUB_PDF' \
 -L -o '/full/path/to/output.pdf'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   295  100   295    0     0    295      0  0:00:01 --:--:--  0:00:01  1666
100   318    0   318    0     0    318      0 --:--:-- --:--:-- --:--:--   521
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
100 2258k    0 2258k    0     0   752k      0 --:--:--  0:00:03 --:--:-- 1295k
